In [1]:
import torch
import torch.nn as nn





In [ ]:
#так будет инициализация
for m in self.modules():
      if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
      elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()
      elif isinstance(m, nn.Linear):
        init.kaiming_normal(m.weight)
        m.bias.data.zero_() 

In [ ]:
from group_norm  import GroupNorm2d #отсюда итпорт https://github.com/chengyangfu/pytorch-groupnormalization
#https://pytorch.org/docs/stable/nn.html
def norm2d(planes, num_channels_per_group=32):
    print("num_channels_per_group:{}".format(num_channels_per_group))
    if num_channels_per_group > 0:
        return GroupNorm2d(planes, num_channels_per_group, affine=True,
                           track_running_stats=False)
    else:
        return nn.BatchNorm2d(planes)


Modda1 =  nn.Sequential(
          nn.Conv2d(3,32,3,padding=2,bias=True)#попробовать elasticnet regularization. что не так с l2
          nn.GroupNorm(3, 32)
          nn.LeakyReLU()
          nn.Conv2d(32,32,3,padding=2,bias=True)
          nn.GroupNorm(3, 32)# https://github.com/chengyangfu/pytorch-groupnormalization это хуячь #вроде так
          nn.LeakyReLU()
          nn.MaxPool2d(2) #даунсемплинг 75 #попробовать fractional max-pooling
          nn.Conv2d(32,64,3,padding=2,bias=True)
          nn.GroupNorm(4, 64)
          nn.LeakyReLU()
          nn.Conv2d(64,64,3,padding=2,bias=True)
          nn.GroupNorm(4, 64)
          nn.LeakyReLU()
          nn.MaxPool2d(3)#даунсемплинг 25
          nn.Conv2d(64,256,3,padding=2,bias=True)
          nn.GroupNorm(4, 256)
          nn.LeakyReLU()
          nn.Conv2d(256,256,3,padding=2,bias=True)
          nn.GroupNorm(4, 256)
          nn.LeakyReLU()
          nn.Conv2d(256,256,3,padding=2,bias=True)
          nn.GroupNorm(4, 256))
          nn.LeakyReLU()
          nn.Conv2d(256,256,3,padding=2,bias=True)
          nn.GroupNorm(4, 256)
          nn.LeakyReLU()
          nn.MaxPool2d(5)#даунсемплинг 5
          nn.Conv2d(256,512,3,padding=2,bias=True)
          nn.GroupNorm(8, 512) 
          nn.LeakyReLU()
          nn.Conv2d(512,512,3,padding=2,bias=True)
          nn.GroupNorm(4, 512)
          nn.LeakyReLU()
          nn.Conv2d(512,512,3,padding=2,bias=True)
          nn.GroupNorm(2, 512)
          nn.LeakyReLU()
          nn.Conv2d(512,512,3,padding=2,bias=True)
          nn.GroupNorm(2, 512)
          nn.LeakyReLU()
          nn.Linear(in_features=5*5*512, out_features=10, bias=True) 
)


In [ ]:
#use pre-trained models (коричневая тетрадь, как хэндлить относительно схожести и количества данных)

In [ ]:
#model zoos

In [ ]:
#в model ensembles/snapshots u can use dropout/stochastic depthdsfvlj

In [ ]:
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)
densenet = models.densenet161(pretrained=True)
inception = models.inception_v3(pretrained=True)

In [ ]:

concatenated = concatenate([Modda1, vgg16,densenet, inception])

In [ ]:
out = Dense(1, activation='softmax', name='output_layer')(concatenated)

merged_model = Model([150, 150], out)
merged_model.compile(loss='binary_crossentropy', optimizer='adam', 
metrics=['accuracy'])

checkpoint = ModelCheckpoint('weights.h5', monitor='val_acc',
save_best_only=True, verbose=2)
early_stopping = EarlyStopping(monitor="val_loss", patience=5)

merged_model.fit([x1, x2], y=y, batch_size=384, epochs=200,
             verbose=1, validation_split=0.1, shuffle=True, 
callbacks=[early_stopping, checkpoint])